# Ordinals

<https://docs.ordinals.com/>

<https://github.com/ordinals/ord/blob/master/bip.mediawiki>

核心是给所有挖出来的聪编号，从0开始。比如第一个block (height = 0)，也就是创世区块，挖出了50个btc，也就是 50_0000_0000 聪，那么这些聪的编号就是聪 0 到 49_9999_9999。

再有就是跟踪这些聪的转账历史，从而记录下每个聪的来龙去脉。

最终就是，对于一个utxo，我们需要知道，对于它所包含的币中的每个聪，其编号是什么。

这里的难点是跟踪聪的转账历史。btc本身并不区分同一个utxo中的所有聪，因此我们需要为其强行赋予一个输入到输出的映射关系。

直观上讲，比如我的一个utxo中包含了编号为1，3，5，7，9的聪，然后我支付3聪给A，1聪作为找零给我自己，还有1聪作为交易手续费给了矿工，那么A将得到1，3，5，我自己得到7，矿工得到9。这个非常符合直观，因此比较好理解。当然，确实有一些corner case需要考虑。

## 几个实验

### 1. 计算一个block挖出来的币的第一个聪的编号

In [216]:
# test data from f2pool.com
block_height = 574993
correct_ord = 1768741250000000

# 每4年减半，也就是没210000个区块减半，初始奖励为50BTC
def get_block_reward(block_height):
    answer = 50 * 10**8 / 2**(block_height // 210000)
    assert answer == int(answer)
    return int(answer)

def get_number_of_the_first_block_satoshi_brute_force(block_height):
    acc = 0
    for height in range(block_height):
        acc += get_block_reward(height)
    return acc

def get_number_of_the_first_block_satoshi(block_height):
    epoch = block_height // 210000 # 第几个4年周期
    acc = 0
    for e in range(epoch): # 每个epoch直接计算 
        acc += 210000 * get_block_reward(e * 210000)
    epoch_start = epoch * 210000
    epoch_offset = block_height - epoch_start
    epoch_reward = get_block_reward(epoch_start)
    acc += epoch_offset * epoch_reward
    return acc

calculated_ord = get_number_of_the_first_block_satoshi(block_height)
calculated_ord_brute_force = get_number_of_the_first_block_satoshi_brute_force(block_height)
assert calculated_ord == calculated_ord_brute_force
print('calculated_ord:', calculated_ord)
print('correct_ord:', correct_ord)
assert calculated_ord == correct_ord
print('OK')

calculated_ord: 1768741250000000
correct_ord: 1768741250000000
OK


### 2. 找到制定编号的聪目前在哪个UTXO中，归谁？

首先找到挖出它的block，然后根据转账历史一个tx一个tx的跟踪，直到找到当前的UTXO，从而找到其当前所有者。

因此一个聪经历的转账次数越多，找到他就越麻烦。比如我们来试着找一下看看。

In [217]:
import os, requests, json

# 测试地址
test_address = '1HtUGfbDcMzTeHWx2Dbgnhc6kYnj1Hp24i'
test_sat = 1358647499999999
test_sat_block = 333458

# from https://ordiscan.com/address/1HtUGfbDcMzTeHWx2Dbgnhc6kYnj1Hp24i/rare-sats we knows this sat belongs to this address

# 首先计算这个sat对应的block
def get_mined_block_number_from_sat(sat):
    epoch = 0
    epoch_start = epoch * 210000
    acc = 0
    epoch_full_reward = 210000 * get_block_reward(epoch_start)
    while acc + epoch_full_reward <= sat:
        acc += epoch_full_reward
        epoch += 1
        epoch_start += 210000
        epoch_full_reward = 210000 * get_block_reward(epoch_start)
    
    epoch_block_reward = get_block_reward(epoch_start)
    epoch_offset = (sat - acc) // epoch_block_reward
    block = epoch_start + epoch_offset
    return block

block = get_mined_block_number_from_sat(test_sat)
print('block:', block)
assert block == test_sat_block

def get_cache_fn_content(fn):
    fn = f'ordinals-data/{fn}'
    # if file does not exists, return None
    if not os.path.exists(fn):
        return None
    with open(fn, 'r') as f:
        return f.read()

def put_cache_fn_content(fn, content):
    fn = f'ordinals-data/{fn}'
    with open(fn, 'w') as f:
        f.write(content)

def get_web_content(url, fn=None, **options):
    if fn != None:
        fn_content = get_cache_fn_content(fn)
        if fn_content is not None:
            return fn_content
    print(f'fetching {url}')
    r = requests.get(url)
    content = None
    if 'allow_404' in options and options['allow_404'] == True:
        content = ''
    else:
        assert r.status_code == 200
        content = r.text
        assert content != None
    if fn != None:
        put_cache_fn_content(fn, content)
    return content

def get_block_hash_from_height(height):
    fn = f'block_hash_{height}.txt'
    url = f'https://mempool.space/api/block-height/{height}'
    hash = get_web_content(url, fn)
    assert len(hash) == 64
    return hash

def get_block_txs_from_height(height):
    hash = get_block_hash_from_height(height)
    url = f'https://mempool.space/api/block/{hash}/txs'
    fn = f'block_txs_{height}.txt'
    content = get_web_content(url, fn)

    obj = json.loads(content)
    assert len(obj) >= 1
    assert obj[0]['vin'][0]['is_coinbase'] == True
    return obj

def get_tx_from_txid(txid):
    assert len(txid) == 64
    url = f'https://mempool.space/api/tx/{txid}'
    fn = f'tx_{txid}.txt'
    content = get_web_content(url, fn)

    obj = json.loads(content)
    assert obj['txid'] == txid
    return obj

# def fill_vin_of_tx(tx, till=None, till_n=None):
#     assert len(tx['txid']) == 64

#     for vin in tx['vin']:
#         if vin['is_coinbase']:
#             vin['prev_tx'] = None
#             continue
#         txid = vin['txid']
#         if txid == till and vin['vout'] == till_n:
#             break
#         vout = vin['vout']
#         assert vin['prevout']

#         prev_tx = get_tx_from_txid(txid)
#         vin['prev_tx'] = prev_tx
    
#     return tx

def get_tx_spent_of_vout(txid, vout):
    assert len(txid) == 64
    assert vout >= 0

    fn = f'tx_spent_at_{txid}_{vout}.txt'
    url = f'https://mempool.space/api/tx/{txid}/outspend/{vout}'
    content = get_web_content(url, fn)
    
    obj = json.loads(content)

    if obj['spent'] == False:
        return None
    
    ret = (obj['txid'], obj['vin'])
    assert len(ret[0]) == 64
    assert ret[1] >= 0

    return ret

def get_tx_fee(txid):
    assert len(txid) == 64
    tx = get_tx_from_txid(txid)

    fee = tx['fee']
    assert fee >= 0
    return fee

    # fill_vin_of_tx(tx)

    # acc = 0
    # for vin in tx['vin']:
    #     assert 'prev_tx' in vin
    #     prev_tx = vin['prev_tx']
    #     value = prev_tx['vout'][vin['vout']]['value']
    #     acc += value
    
    # for vout in tx['vout']:
    #     acc -= vout['value']
    
    # assert acc >= 0

    # return acc

assert get_tx_fee('6db71e70bd0e5b9f2cc0678ff2bdb47215e2adc68dc72bdac973d93c76a54b7b') == 130_0000

def get_vout_for_offset(tx, offset):
    assert offset >= 0
    acc = 0
    idx = 0
    for vout in tx['vout']:
        if acc + vout['value'] > offset:
            return (idx, vout, offset - acc)
        acc += vout['value']
        idx += 1
    return None

def trace_sat(pos):
    txid = pos[0]
    assert len(txid) == 64
    offset = pos[1] # offset in all output sat's
    assert offset >= 0
    tx = get_tx_from_txid(txid)

    # find the output
    acc = 0
    tmp = get_vout_for_offset(tx, offset)

    correct_vout = None
    correct_n = 0
    correct_offset = 0
    if tmp != None:
        correct_n = tmp[0]
        correct_vout = tmp[1]
        correct_offset = tmp[2]
        assert correct_n >= 0
        assert correct_offset >= 0
    
    if tx['vin'][0]['is_coinbase'] == True and correct_vout == None:
        # destroyed
        print('destroyed')
        return None
    
    # not fee
    if correct_vout != None:
        spent_at = get_tx_spent_of_vout(txid, correct_n)
        if spent_at == None:
            # not yet spent
            return None

        spent_at_txid = spent_at[0]
        spent_at_vin_idx = spent_at[1]

        spent_at_tx = get_tx_from_txid(spent_at_txid)

        acc = 0
        vin_found = False
        for vin in spent_at_tx['vin']:
            if vin['txid'] == txid and vin['vout'] == correct_n:
                vin_found = True
                break

            assert 'prevout' in vin
            prevout_value = vin['prevout']['value']
            assert prevout_value >= 0

            acc += prevout_value
        assert vin_found == True

        return (spent_at_txid, acc + correct_offset)
    
    #print('spent as fee')

    assert tx['status']['confirmed'] == True

    block_height = tx['status']['block_height']
    assert block_height >= 0
    block_hash = tx['status']['block_hash']
    assert len(block_hash) == 64

    block_reward = get_block_reward(block_height)
    txs = get_block_txs_from_height(block_height)

    coinbase_txid = txs[0]['txid']
    
    for tx in txs[1:]: # skip the coinbase tx
        if tx['txid'] == txid:
            break
        fee_of_this_tx = tx['fee']
        assert fee_of_this_tx >= 0
        block_reward += fee_of_this_tx
    
    return (coinbase_txid, block_reward + correct_offset)

def find_utxo_of_sat(sat):
    print(f'getting utxo for sat {sat}')
    block = get_mined_block_number_from_sat(sat)
    print(f'block is {block}')

    tx_list = get_block_txs_from_height(block)
    coinbase_tx = tx_list[0]

    # current position of this sat is defined as (txid, output_offset)
    cur_pos = (coinbase_tx['txid'], test_sat - get_number_of_the_first_block_satoshi(test_sat_block))

    n_transfer = 0
    tx = get_tx_from_txid(cur_pos[0])
    while True:
        assert tx['status']['confirmed'] == True
        block_height = tx['status']['block_height']
        is_coinbase = tx['vin'][0]['is_coinbase']
        vout = get_vout_for_offset(tx, cur_pos[1])
        vout_idx = None
        vout_offset = None
        if vout != None:
            vout_idx = vout[0]
            vout_offset = vout[2]
        else:
            vout_idx = 'as_fee'
            vout_offset = 'as_fee'
        print(f'tx = {cur_pos[0]}, vout = {vout_idx}, offset = {vout_offset}, block = {block_height}, n_transfer = {n_transfer}, coinbase = {is_coinbase}')
        next_pos = trace_sat(cur_pos)
        if next_pos == None:
            break
        tx = get_tx_from_txid(next_pos[0])
        if tx['status']['confirmed'] == False:
            break
        cur_pos = next_pos
        n_transfer += 1

    tx = get_tx_from_txid(cur_pos[0])
    vout = get_vout_for_offset(tx, cur_pos[1])
    assert vout != None
    vout_idx = vout[0]
    vout = vout[1]
    addr = vout['scriptpubkey_address']
    return (cur_pos[0], vout_idx, addr)

ret = find_utxo_of_sat(test_sat)
print(ret)
assert ret[2] == test_address


block: 333458
getting utxo for sat 1358647499999999
block is 333458
tx = 0583d72b6e68b891832be447d39826201638fc0f113bb1e3febd288e05c299e2, vout = 0, offset = 2499999999, block = 333458, n_transfer = 0, coinbase = True
tx = 3cdb5647417fa1fc42cbf545491b7663382c2c839e93e80bc34ca5b8d91d4dfd, vout = 4, offset = 671925684, block = 333580, n_transfer = 1, coinbase = False
('3cdb5647417fa1fc42cbf545491b7663382c2c839e93e80bc34ca5b8d91d4dfd', 4, '1HtUGfbDcMzTeHWx2Dbgnhc6kYnj1Hp24i')


### 3. 计算一个UTXO中包含的聪的所有编号？

一个当前的UTXO，表示的是一个到现在为止还没有被花费掉的 TX OUTPUT。

一个历史的UTXO，表示的是历史上该 TX OUTPUT没有被花费掉，后来某个时刻被花费掉了。

一个UTXO包含了多少聪，就有多少个聪编号。比如一个UTXO包含了一个btc，也就是 $ 10^8 $ 聪，那就是 $10^8$ 个编号。一一列举出这些编号，往往是复杂的。

不过，对于某些UTXO，这事儿还是简单，比如一个空块儿的coinbase的output。这些output的聪编号是连续的，所以知道了第一个聪的编号，就能直接知道所有聪的编号。这里说到空块儿，是因为如果不是空块儿，那么coinbase的output中就会包含所有交易的手续费中包含的聪，那这些聪编号就复杂了。

如果没有提前索引数据，计算一个utxo中的所有聪的编号，就需要从交易的vin不断的回溯，直到找到所有相关的coinbase为止，这样才能确定该utxo中的所有聪的编号，这往往会牵涉到很多很多的交易，非常的复杂。记住，coinbase中的聪部分来自挖矿，部分来自fee，fee的部分就更加的复杂了。

但是如果提前做了索引，那就可以增量的处理每个区块。说是这样说，据说为了能够实时的计算所有UTXO（当前的和历史的），索引数据量需要 10T 级别。如果只计算当前UTXO，需要 100G 级别的索引数据。（数据来自 BIP ）。

## 铭文

既然每个聪都有了编号，那么聪就有点类似NFT了，NFT得有些附带的数据才行，比如一个图片之类的。

于是，就有了给一个聪搞点数据的需求。

据说是考虑到btc字节码的一些数据量的限制（ @FIXME ），ORD协议选择了用 P2TR 支付方式来把数据搞到链上去。

具体拿一个例子来分析。

In [218]:
commit_txid = '93d6c93ac577ba9c11636cd9ed57133dcd49666b1fac7657065c1e9d46257766'
tx = get_tx_from_txid(commit_txid)
vout = tx['vout'][0]
assert vout['scriptpubkey_type'] == 'v1_p2tr'
vout_script_pubkey = vout['scriptpubkey_asm']
print(f'vout_script_pubkey = {vout_script_pubkey}')


vout_script_pubkey = OP_PUSHNUM_1 OP_PUSHBYTES_32 2d90edea641674ac4b7f2eac7d0b737fc1eab5e99513382ceb8e98266a796163


在这个所谓的commit tx中，核心是一个hash为 **2d90edea641674ac4b7f2eac7d0b737fc1eab5e99513382ceb8e98266a796163** 的脚本。

In [219]:
reveal_txid = '3d59bc1eef840222dc4a1e4ae3ed4d7d4de7a6ba66c55b2537d2fbeb6349c047'
tx = get_tx_from_txid(reveal_txid)
witness = tx['vin'][0]['witness']
witness_json = json.dumps(witness, indent=2)
print(f'witness = {witness_json}')

inner_witnessscript_asm = tx['vin'][0]['inner_witnessscript_asm']
inner_witnessscript_asm = inner_witnessscript_asm.split(' ')
inner_witnessscript_asm_json = json.dumps(inner_witnessscript_asm, indent=2)
print(f'inner_witnessscript_asm = {inner_witnessscript_asm_json}')


witness = [
  "79edae917792268e6ac0567ce28195bda67f1afff9d2b8514d6daca390a05be79091bebda71af36bbdd6327666dcaec875999754f24e0c8c83c07b18da4c1ba501",
  "51690063036f7264010117746578742f68746d6c3b636861727365743d7574662d38004d08023c73637269707420646174612d733d2230783166353233616463666562393261376632316137326530366264633963643963323236356539306630623234666262613335363035326162656564306633316322207372633d222f636f6e74656e742f663830623933343636613238633565666337303366616230326265656262663465333265316263346630363361633237666564666437396164393832663263656930223e3c2f7363726970743e3c626f6479207374796c653d22646973706c61793a206e6f6e65223e3c2f626f64793e000000000000000062766d76341b4504c02f0b784334f50b8197dae27b4d8888a14784f0270dd5a513be207b2f10d93a008e0e6709077663fd9bfda68412ce3cf03c5bc36a8d23464f03792c510303fcca73140e10129f9e3983a4dd9f9e062de190d2c8a380134ae9681789beef7ca2e381ce873ad9bdcd0859165ef083adaddf94ac168b71717c5ad123985d08195dbbfd2700af9e23d7368bf3252a01189c80d1906f7da2c49ea1c5acf0a4ef10aa8

这里最核心的，是真正的 TAPROOT SCRIPT，也就是 **51** 打头的这个东西，也就是上面的 **inner_witnessscript_asm** 中的字节码。

In [220]:
op_list = inner_witnessscript_asm[4:-1]

def decode_inscription(op_list):
    stack = []
    idx = 0
    while idx < len(op_list):
        op = op_list[idx]
        if op == 'OP_0':
            stack.append('00')
            idx += 1
            continue
        if op.startswith('OP_PUSHNUM_'):
            num_str = op[len('OP_PUSHNUM_'):]
            num = int(num_str)
            assert num >= 0 and num <= 16
            stack.append(f'{num:02x}')
            idx += 1
            continue
        if op.startswith('OP_PUSH'):
            stack.append(op_list[idx + 1])
            idx += 2
            continue
        assert False

    assert bytes.fromhex(stack[0]).decode() == 'ord'
    stack = stack[1:]

    meta = {}
    body = bytes(0)
    idx = 0
    while idx < len(stack):
        key = bytes.fromhex(stack[idx])
        assert len(key) == 1
        key_i = int(key[0])
        if key_i == 0:
            idx += 1
            while idx < len(stack):
                body += bytes.fromhex(stack[idx])
                idx += 1
            break
        value = stack[idx + 1]
        key_name = 'unknown_' + str(key_i)
        as_str = False
        match key_i:
            case 1:
                key_name = 'content-type'
                as_str = True
            case 2:
                key_name = 'pointer'
            case 3:
                key_name = 'parent'
            case 5:
                key_name = 'metadata'
            case 7:
                key_name = 'metaprotocol'
            case 9:
                key_name = 'content_encoding'
                as_str = True
            case 11:
                key_name = 'delegate'
        if as_str == True:
            value = bytes.fromhex(value).decode()
        meta[key_name] = value
        idx += 2

    return (meta, body)

print(json.dumps(meta, indent=2))
print(body)


{
  "content-type": "text/plain;charset=utf-8"
}
{"p":"brc-20","op":"transfer","tick":"ordi","amt":"900"}


这个和某个聪捆绑的东西就是 **铭文** ，铭文可以用两个东西来index，一个是reveal tx的txid，一个是在这个tx的所有input中reveal出来的顺序。

注意，一个tx的一个input中可以搞出多个铭文，他们默认对应这个input的第一个聪，除非铭文里面写了 **pointer** ，从而刻意指定了哪个聪。

因此，上面这个铭文的编号就是

**3d59bc1eef840222dc4a1e4ae3ed4d7d4de7a6ba66c55b2537d2fbeb6349c047i0** ，其中的 **i** 之前是  txid，**i** 之后是index。

而且，按照所有铭文搞出来的顺序，每个铭文还有一个唯一编号。比如上面这个叫做 <https://ordiscan.com/inscription/70316566> ，也就是编号为 **70316566**。 (@FIXME ，官网说这里有个bug，对于一些特殊的铭文，其编号顺序有问题）

ord官方还提供了一个地方来显示铭文的内容

<https://ordiscan.com/content/3d59bc1eef840222dc4a1e4ae3ed4d7d4de7a6ba66c55b2537d2fbeb6349c047i0>


## BRC-20

brc-20又是什么鬼，看起来类似 erc20 的东西，但是这个东西又是基于 ordinals 来搞的，有些奇怪。

<https://ordiswap.gitbook.io/ordiswap/protocol-concepts/brc-20-standard>

直接来一个例子，也就是 ORDI 这个 brc20，其对应一个铭文

<https://ordinals.com/inscription/b61b0172d95e266c18aea0c624db987e971a5d6d4ebc2aaed85da4642d635735i0>

其reveal tx是 <https://mempool.space/zh/tx/b61b0172d95e266c18aea0c624db987e971a5d6d4ebc2aaed85da4642d635735>

In [221]:
reveal_txid = 'b61b0172d95e266c18aea0c624db987e971a5d6d4ebc2aaed85da4642d635735'

def print_inscription(reveal_txid, splice_start, splice_end=-1, print_body=True):
    tx = get_tx_from_txid(reveal_txid)

    inner_witnessscript_asm = tx['vin'][0]['inner_witnessscript_asm']
    inner_witnessscript_asm = inner_witnessscript_asm.split(' ')
    inner_witnessscript_asm_json = json.dumps(inner_witnessscript_asm, indent=2)
    print(f'inner_witnessscript_asm = {inner_witnessscript_asm_json}')

    op_list = inner_witnessscript_asm[splice_start:splice_end]
    print(f'op_list = {op_list}')
    meta, body = decode_inscription(op_list)
    if print_body:
        print(json.dumps(meta, indent=2))
        print(body.decode())
    else:
        return (meta,body)


print_inscription(reveal_txid, 5)


inner_witnessscript_asm = [
  "OP_PUSHBYTES_32",
  "9e2849b90a2353691fccedd467215c88eec89a5d0dcf468e6cf37abed344d746",
  "OP_CHECKSIG",
  "OP_0",
  "OP_IF",
  "OP_PUSHBYTES_3",
  "6f7264",
  "OP_PUSHBYTES_1",
  "01",
  "OP_PUSHBYTES_24",
  "746578742f706c61696e3b636861727365743d7574662d38",
  "OP_0",
  "OP_PUSHDATA1",
  "7b200a20202270223a20226272632d3230222c0a2020226f70223a20226465706c6f79222c0a2020227469636b223a20226f726469222c0a2020226d6178223a20223231303030303030222c0a2020226c696d223a202231303030220a7d",
  "OP_ENDIF"
]
op_list = ['OP_PUSHBYTES_3', '6f7264', 'OP_PUSHBYTES_1', '01', 'OP_PUSHBYTES_24', '746578742f706c61696e3b636861727365743d7574662d38', 'OP_0', 'OP_PUSHDATA1', '7b200a20202270223a20226272632d3230222c0a2020226f70223a20226465706c6f79222c0a2020227469636b223a20226f726469222c0a2020226d6178223a20223231303030303030222c0a2020226c696d223a202231303030220a7d']
{
  "content-type": "text/plain;charset=utf-8"
}
{ 
  "p": "brc-20",
  "op": "deploy",
  "tick": "ordi",
  "max": "210000

上面的 **lim** 的意思是，每次mint至多这么多个，防止被人包干了。

于是乎，这个铭文就对应了一个brc20的deploy？那这个brc20就有了？从虚空中产生了？

看一个 mint ORDI 的例子

<https://ordinals.com/inscription/778bf74299ba8b29df3fcf22ce66cdc45a87c21cc229eb0f6d86bc57539971d2i0>


In [222]:
reveal_txid = '778bf74299ba8b29df3fcf22ce66cdc45a87c21cc229eb0f6d86bc57539971d2'

print_inscription(reveal_txid, 5)


inner_witnessscript_asm = [
  "OP_PUSHBYTES_32",
  "81b7537cf86b4fe88c7320765a3c126c1734f2dcd2504585deaf50a5430a4da6",
  "OP_CHECKSIG",
  "OP_0",
  "OP_IF",
  "OP_PUSHBYTES_3",
  "6f7264",
  "OP_PUSHBYTES_1",
  "01",
  "OP_PUSHBYTES_24",
  "746578742f706c61696e3b636861727365743d7574662d38",
  "OP_0",
  "OP_PUSHBYTES_71",
  "7b200a20202270223a20226272632d3230222c0a2020226f70223a20226d696e74222c0a2020227469636b223a20226f726469222c0a202022616d74223a202231303030220a7d",
  "OP_ENDIF"
]
op_list = ['OP_PUSHBYTES_3', '6f7264', 'OP_PUSHBYTES_1', '01', 'OP_PUSHBYTES_24', '746578742f706c61696e3b636861727365743d7574662d38', 'OP_0', 'OP_PUSHBYTES_71', '7b200a20202270223a20226272632d3230222c0a2020226f70223a20226d696e74222c0a2020227469636b223a20226f726469222c0a202022616d74223a202231303030220a7d']
{
  "content-type": "text/plain;charset=utf-8"
}
{ 
  "p": "brc-20",
  "op": "mint",
  "tick": "ordi",
  "amt": "1000"
}


这样就 mint 了 1000 个 ORDI 了。貌似这个铭文和 deploy 的那个铭文毛关系都没有，只是 BRC20 客户端自己进行规则解释的结果？

下面是一个 transfer 的例子

In [223]:
reveal_txid = 'be412e058204fa0d9ac4babd2d73bd1d148c88bc904817d9882026a5509c9e06'

print_inscription(reveal_txid, 5)


inner_witnessscript_asm = [
  "OP_PUSHBYTES_32",
  "72105d510c324751598b1b0c54a0ad002a1dd6a2970b4345cb42d12577233018",
  "OP_CHECKSIG",
  "OP_0",
  "OP_IF",
  "OP_PUSHBYTES_3",
  "6f7264",
  "OP_PUSHBYTES_1",
  "01",
  "OP_PUSHBYTES_24",
  "746578742f706c61696e3b636861727365743d7574662d38",
  "OP_0",
  "OP_PUSHBYTES_56",
  "7b2270223a226272632d3230222c226f70223a227472616e73666572222c227469636b223a226f726469222c22616d74223a22393030227d",
  "OP_ENDIF"
]
op_list = ['OP_PUSHBYTES_3', '6f7264', 'OP_PUSHBYTES_1', '01', 'OP_PUSHBYTES_24', '746578742f706c61696e3b636861727365743d7574662d38', 'OP_0', 'OP_PUSHBYTES_56', '7b2270223a226272632d3230222c226f70223a227472616e73666572222c227469636b223a226f726469222c22616d74223a22393030227d']
{
  "content-type": "text/plain;charset=utf-8"
}
{"p":"brc-20","op":"transfer","tick":"ordi","amt":"900"}


还有这种所谓 inscription_transfer 的。这种直接给了地址，而不是用utxo的地址？

In [224]:
reveal_txid = 'ed6d6c678dd5c7669bac7539c79c3cd74854a946c1870ee890116f87b8ea9c2c'

print_inscription(reveal_txid, 5)

inner_witnessscript_asm = [
  "OP_PUSHBYTES_32",
  "08634c1af665c7e94ab7365e26bbaa92339cee5ed16952d9fbce6d003d70e7cb",
  "OP_CHECKSIG",
  "OP_0",
  "OP_IF",
  "OP_PUSHBYTES_3",
  "6f7264",
  "OP_PUSHBYTES_1",
  "01",
  "OP_PUSHBYTES_16",
  "6170706c69636174696f6e2f6a736f6e",
  "OP_0",
  "OP_PUSHDATA1",
  "7b0a202020202270223a20226272632d3230222c0a20202020226f70223a20227472616e73666572222c0a20202020227469636b223a20226f726469222c0a2020202022616d74223a202231222c0a2020202022746f223a2022626331706d7870337235677a346e7563307779366779776e6a736a39776576736a6c64713279646b396136306877777461766877353861716d7239687a30222c0a2020202022666565223a202231220a7d",
  "OP_ENDIF"
]
op_list = ['OP_PUSHBYTES_3', '6f7264', 'OP_PUSHBYTES_1', '01', 'OP_PUSHBYTES_16', '6170706c69636174696f6e2f6a736f6e', 'OP_0', 'OP_PUSHDATA1', '7b0a202020202270223a20226272632d3230222c0a20202020226f70223a20227472616e73666572222c0a20202020227469636b223a20226f726469222c0a2020202022616d74223a202231222c0a2020202022746f223a2022626331706d

<https://layer1.gitbook.io/layer1-foundation/protocols/brc-20/documentation>

这个页面解释了铭文和BRC20的关系

1. mint而言，搞出来的铭文的第一个owner得到balance，以后这个铭文转移到任何地方都没有关系了。

2. transfer而言，搞出来的铭文transfer给sender，然后sender transfer这个铭文的目标得到balance。

这尼玛感觉有个bug，举例如下：

1. A有比如 1000 个 ORDI

2. B搞一个transfer的铭文，内容是转移出500个，然后reveal给A，这个时候A并不知道，反正就是被动接受了而已

3. A往外打钱，在不知情的情况下，把B打给自己的UTXO给打出去了，这样 500 个ORDI就不知情的打出去了？


不过据说 brc20 钱包对这种bug有提示，在你转出币的时候，如果也有 brc20 转出，应该会提示你 **危险** 吧？

另外brc20这一套东西，感觉是偷懒的结果，就是初期严格依赖于 ordinals 的基础设施，自己不愿意搭建，所以连一个 transfer 竟然都需要3个交易来完成，太浪费了。

## 符文 Runes

<https://docs.ordinals.com/runes.html>

我感觉符文出来最大的原因就是 BRC20 太笨重了， Ordinals 创始人看不下去了：）

Runes 使用了 OP_RETURN 这个东东。

<https://en.bitcoin.it/wiki/OP_RETURN>

我们直接找一个 runes 来看看，随便找了一个 **BITCOIN•PEPE•MATRIX**。


In [268]:
import leb128

deploy_txid = '6ea0f940a1386cbb146e134933ca3533c0759101a46744d0b207a283f5f57b23'

assert leb128.u.encode(624485) == bytearray([0xe5, 0x8e, 0x26])

def get_rune_name(num):
    data = bytes(0)
    while num > 0:
        x = num % 26
        data = bytes([x - 1 + 65]) + data # FIXME ...，怎么差了一个 1 呢，而且最后一个字母感觉有问题
        num //= 26
    return data.decode()

def decode_leb128_list(data):
    ret = []
    while len(data) > 0:
        idx = 0
        while data[idx] >= 0x80:
            assert idx < len(data) - 1
            idx += 1
        num = leb128.u.decode(data[:idx + 1])
        ret.append(num)
        data = data[idx + 1:]
    return ret

def get_flag_name(flag):
    match flag:
        case 0: return 'Body'
        case 2: return 'Flags'
        case 4: return 'Rune'
        case 6: return 'Premine'
        case 8: return 'Cap'
        case 10: return 'Amount'
        case 12: return 'HeightStart'
        case 14: return 'HeightEnd'
        case 16: return 'OffsetStart'
        case 18: return 'OffsetEnd'
        case 20: return 'Mint'
        case 22: return 'Pointer'
        case 126: return 'Cenotaph'
        
        case 1: return 'Divisibility'
        case 3: return 'Spacers'
        case 5: return 'Symbol'
        case 127: return 'Nop'

    return 'unknwon_' + str(flag)

def decode_num_list_as_untyped_msg(num_list):
    fields = {}
    edicts = [] # list of {block,tx_idx,amount,output}

    idx = 0
    while idx < len(num_list):
        x = num_list[idx]
        idx += 1
        if x != 0:
            assert idx < len(num_list)
            y = num_list[idx]
            if x in fields:
                fields[x].append(y)
            else:
                fields[x] = [y]
            idx += 1
            continue
        num_list = num_list[idx:]
        assert len(num_list) % 4 == 0
        while len(num_list) > 0:
            edicts.append(num_list[:4])
            num_list = num_list[4:]
        break
    return fields, edicts

def decode_runes_op(txid):
    tx = get_tx_from_txid(txid)

    print(f'decode_runes_op for {txid}')
    print('')
    is_first = True
    for vout in tx['vout']:
        if vout['scriptpubkey_type'] != 'op_return':
            continue
        scriptpubkey_asm = vout['scriptpubkey_asm']
        op_list = scriptpubkey_asm.split(' ')

        # Runes协议要求的
        if op_list[0] != 'OP_RETURN':
            continue
        if op_list[1] != 'OP_PUSHNUM_13':
            continue

        op_list = op_list[2:]

        data = bytes(0)
        idx = 0
        while idx < len(op_list):
            op = op_list[idx]

            if op.startswith('OP_PUSHBYTES_'):
                idx += 1
                assert idx < len(op_list)
                next_data = op_list[idx]
                next_data = bytes.fromhex(next_data)
                data += next_data

                idx += 1
                continue

            assert False
        
        if not is_first:
            print('-' * 50)
        num_list = decode_leb128_list(data)
        print(num_list)

        fields, edicts = decode_num_list_as_untyped_msg(num_list)
        #print(f'fields = {fields}')
        fields_with_name = {}
        for x in fields:
            x_name = get_flag_name(x)
            y = fields[x]
            fields_with_name[x_name] = y
            print(f'field: {x:3d} , name: {x_name:12s} , value: {y}')
        print(f'edicts = {edicts}')

        if 'Flags' in fields_with_name:
            sub_list = fields_with_name['Flags']
            assert len(sub_list) == 1
            x = sub_list[0]
            if (x & 1) != 0:
                print('Flag.Etching = True')
            if (x & 2) != 0:
                print('Flag.Terms = True')
            if (x & 4) != 0:
                print('Flag.Turbo = True')
            if x == 127:
                print('Flag = Cenotaph')

        if 'Rune' in fields_with_name:
            sub_list = fields_with_name['Rune']
            assert len(sub_list) == 1
            print('Rune name is @FIXME ', get_rune_name(sub_list[0]))
        
        if_first = False

decode_runes_op(deploy_txid)



decode_runes_op for 6ea0f940a1386cbb146e134933ca3533c0759101a46744d0b207a283f5f57b23

[2, 7, 4, 103611929363950885954073, 1, 0, 3, 1088, 5, 8473, 10, 69420, 8, 100001, 12, 840269, 14, 841420, 22, 1]
field:   2 , name: Flags        , value: [7]
field:   4 , name: Rune         , value: [103611929363950885954073]
field:   1 , name: Divisibility , value: [0]
field:   3 , name: Spacers      , value: [1088]
field:   5 , name: Symbol       , value: [8473]
field:  10 , name: Amount       , value: [69420]
field:   8 , name: Cap          , value: [100001]
field:  12 , name: HeightStart  , value: [840269]
field:  14 , name: HeightEnd    , value: [841420]
field:  22 , name: Pointer      , value: [1]
edicts = []
Flag.Etching = True
Flag.Terms = True
Flag.Turbo = True
Rune name is @FIXME  BITCOINPEPEMATRIW


In [275]:
mint_txid = '1b8587082a46af5f1cde323f56bcf22c586fc9ad8fc314d71e888abb2108c24a'

decode_runes_op(mint_txid)

fetching https://mempool.space/api/tx/1b8587082a46af5f1cde323f56bcf22c586fc9ad8fc314d71e888abb2108c24a
decode_runes_op for 1b8587082a46af5f1cde323f56bcf22c586fc9ad8fc314d71e888abb2108c24a

[20, 840000, 20, 1]
field:  20 , name: Mint         , value: [840000, 1]
edicts = []


transfer呢？

研究了半天，感觉所有Runes是寄生在 UTXO 里面的，正常转账就会让 Runes 转走。于是，就可以一下子转走比如很多种币。也可以同时用 OP_RETURN 来改变其行为。

比如这个

<https://www.oklink.com/zh-hans/btc/tx/1b8587082a46af5f1cde323f56bcf22c586fc9ad8fc314d71e888abb2108c24a/transfer>

一大堆的符文转账在里面。